In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import tensorflow as tf
#import  format_output
from datetime_extension import timedate_process
from produce_rnn_flow import aggregate_time
from math import sqrt

In [2]:
#read block data
df=pd.read_csv("blocks.csv",header=0)
#print(df.shape)
df=timedate_process(df)
df=aggregate_time(df)
df.head()


['.ipynb_checkpoints', 'blocks.csv', 'checkpoint', 'datetime_extension.py', 'LSTM (1).ipynb', 'LSTM_new.ipynb', 'prediction.model-1.data-00000-of-00001', 'prediction.model-1.index', 'prediction.model-1.meta', 'produce_rnn_flow.py', 'weather', 'yuan.ipynb', '__pycache__']


,Level,latBlock,lngBlock,month,day,h7,h19,h22,h10,h20,...,h15_neighbor2,h16_neighbor2,h17_neighbor2,h18_neighbor2,h19_neighbor2,h20_neighbor2,h21_neighbor2,h22_neighbor2,h23_neighbor2,days
0,0,0,20,4,1,0,0,0,1,0,...,24.5,80.0,20.0,3.0,1.0,1.0,1.0,1.0,0.0,0
1,0,0,20,4,2,0,0,0,0,0,...,16.0,16.5,35.5,2.5,1.5,1.5,2.0,1.5,0.5,1
2,0,0,20,4,3,0,0,0,1,0,...,34.0,4.5,7.5,8.5,1.5,1.5,1.5,1.5,0.5,2
3,0,0,20,4,5,0,0,0,1,0,...,6.5,30.5,29.5,0.0,4.5,44.5,21.5,14.5,31.5,4
4,0,0,20,4,6,0,0,0,1,0,...,16.5,28.5,9.0,12.5,5.5,9.0,1.5,1.5,0.5,5


In [4]:
#print(max(df[(level=='0')].latBlock))
df=df[(df.Level==0) & (df.latBlock==0) & (df.lngBlock==20)]
df.drop(['Level','latBlock','lngBlock','month','day'], axis=1,inplace=True)
print(df)
print(len(df))

    h7  h19  h22  h10  h20  h12  h15  h8  h13  h11  ...   h15_neighbor2  \
0    0    0    0    1    0    0    0   0    0    0  ...            24.5   
1    0    0    0    0    0    0    0   0    0    0  ...            16.0   
2    0    0    0    1    0    0    0   0    0    0  ...            34.0   
3    0    0    0    1    0    1    0   0    1    0  ...             6.5   
4    0    0    0    1    0    0    0   0    2    0  ...            16.5   
5    0    0    0    0    0    0    1   0    0    0  ...            10.0   
6    0    0    0    0    0    0    0   0    1    0  ...            19.0   
7    0    0    0    0    0    0    1   0    0    0  ...            17.0   
8    0    0    0    0    0    0    0   0    0    0  ...             4.0   
9    0    0    0    0    0    0    0   0    0    1  ...            10.0   
10   0    0    0    0    0    2    0   0    0    0  ...             9.5   
11   0    0    0    0    0    0    0   0    0    0  ...            17.5   
12   0    0    0    0    

In [5]:
data=[]
for i in range(0,len(df)):
    data.append(df.iloc[i].tolist())
print(data)
data=np.array(data)

[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.5, 0.0, 0.0, 0.0, 2.5, 1.5, 2.0, 2.5, 1.5, 4.0, 4.5, 3.5, 5.0, 3.5, 3.5, 2.5, 1.0, 1.5, 1.5, 1.5, 0.5, 2.0, 0.5

In [6]:
# define parameter
rnn_unit = 10  # hidden layer units
input_size = 73
output_size = 24
lr = 0.0006  # learning rate
N = 20 #ecope
N_model = 2 #each N time save model
length = 0
# ——————————————————input data——————————————————————
length=range(len(data))
#print(data)
# get train set
def get_train_data(batch_size=2, time_step=1, train_begin=0, train_end=length):
    batch_index = []
    data_train = data[train_begin:train_end]
    normalized_train_data = data_train
    #print(normalized_train_data)#
    train_x, train_y = [], []  # train set
    for i in range(len(normalized_train_data) - time_step):
        if i % batch_size == 0:
            batch_index.append(i)
        x = normalized_train_data[i:i + time_step, :73]
        y = normalized_train_data[i:i + time_step, 0:24]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data) - time_step))
    #print(train_y)
    return batch_index, train_x, train_y


# test set
def get_test_data(time_step=1, test_begin=0):
    data_test = data[test_begin:]
    mean = np.mean(data_test, axis=0)
    std = np.std(data_test, axis=0)

    normalized_test_data = (data_test - mean) / std  # normalized
    size = (len(normalized_test_data) + time_step - 1) // time_step  # 
    test_x, test_y = [], []
    for i in range(size - 1):
        x = normalized_test_data[i * time_step:(i + 1) * time_step, :73]
        y = normalized_test_data[i * time_step:(i + 1) * time_step, 0:24]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i + 1) * time_step:, :73]).tolist())
    test_y.extend((normalized_test_data[(i + 1) * time_step:, 0:24]).tolist())
    return mean, std, test_x, test_y




weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    #'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
    'out': tf.Variable(tf.random_normal([rnn_unit, output_size]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    #'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
    'out': tf.Variable(tf.constant(0.1, shape=[output_size, ]))
}



def lstm(X):
    batch_size = tf.shape(X)[0]
    time_step = tf.shape(X)[1]
    w_in = weights['in']
    b_in = biases['in']
    input = tf.reshape(X, [-1, input_size])  
    input_rnn = tf.matmul(input, w_in) + b_in
    input_rnn = tf.reshape(input_rnn, [-1, time_step, rnn_unit]) 
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    output_rnn, final_states = tf.nn.dynamic_rnn(cell, input_rnn, initial_state=init_state,
                                                 dtype=tf.float32)  
    output = tf.reshape(output_rnn, [-1, rnn_unit])  
    w_out = weights['out']
    b_out = biases['out']
    pred = tf.matmul(output, w_out) + b_out
    return pred, final_states



def train_lstm(batch_size=2, time_step=1, train_begin=0, train_end=10):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None, time_step, output_size])
    batch_index, train_x, train_y = get_train_data(batch_size, time_step, train_begin, train_end)
    with tf.variable_scope("sec_lstm"):
        pred, _ = lstm(X)

    loss = tf.reduce_mean(tf.square(tf.reshape(pred, [-1,output_size]) - tf.reshape(Y, [-1,output_size])))
    train_op = tf.train.AdamOptimizer(lr).minimize(loss)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
    with tf.Session() as sess:
        try:
            module_file = tf.train.latest_checkpoint('.')
            saver.restore(sess, module_file)
        except:
            sess.run(tf.global_variables_initializer())
        # 重复训练N次
        for i in range(N):
            for step in range(len(batch_index) - 1):
                print('X:$$$$$$$$$$$$')
                print(train_x[batch_index[step]:batch_index[step + 1]])
                print('Y:$$$$$$$$$$$$')
                print(train_y[batch_index[step]:batch_index[step + 1]])
                _, loss_ = sess.run([train_op, loss], feed_dict={X: train_x[batch_index[step]:batch_index[step + 1]],
                                                                 Y: train_y[batch_index[step]:batch_index[step + 1]]})
            #print(i, loss_)
            if i % N_model:
                print("save model：", saver.save(sess, '.\prediction.model', global_step=i))

train_lstm()






INFO:tensorflow:Restoring parameters from .\prediction.model-1
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 

save model： .\prediction.model-1
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

save model： .\prediction.model-3
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

save model： .\prediction.model-5
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

save model： .\prediction.model-7
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

...

save model： .\prediction.model-11
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

save model： .\prediction.model-13
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

save model： .\prediction.model-15
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

save model： .\prediction.model-17
X:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 4.0, 8.5, 3.5, 6.0, 4.0, 6.5, 13.0, 5.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 10.5, 31.0, 5.0, 23.0, 32.0, 24.5, 80.0, 20.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.5, 6.5, 7.0, 6.5, 10.0, 7.0, 6.5, 9.5, 1.0, 1.0, 0.5, 1.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 7.0, 14.5, 27.5, 29.0, 33.0, 16.0, 16.5, 35.5, 2.5, 1.5, 1.5, 2.0, 1.5, 0.5, 1.0]]]
Y:$$$$$$$$$$$$
[[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

save model： .\prediction.model-19


In [7]:
def prediction(time_step=1):
    X = tf.placeholder(tf.float32, shape=[None, time_step, input_size])
    Y = tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    mean, std, test_x, test_y = get_test_data(time_step)
    with tf.variable_scope("sec_lstm", reuse=True):
        pred, _ = lstm(X)
    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        module_file = tf.train.latest_checkpoint('.')
        saver.restore(sess, module_file)
        
        
        test_predict = []
        for step in range(len(test_x) - 1):
            prob = sess.run(pred, feed_dict={X: [test_x[step]]})
            predict = prob.reshape((-1,1))
            test_predict.extend(predict)
        predict_test=[]
        for i in range(len(test_predict)):
            if i%24==0+1:
                predict_test.append(test_predict[i])
        #print(predict_test)
        y_test=[]
        test_y=np.array(test_y).reshape(-1,1)
        for i in range(len(test_y)):
            if i%24==0+1:
                y_test.append(test_y[i])
        test_y=np.array(y_test)
        test_predict=np.array(predict_test)
        test_y=np.nan_to_num(test_y)
        test_predict=np.nan_to_num(test_predict)
        test_y_result=[]
        test_predict_result=[]
        for i in test_y:
            test_y_result.append(i[0])
        for i in test_predict:
            test_predict_result.append(i[0])    
        test_predict_result=np.array(test_predict_result)
        test_y_result=np.array(test_y_result)
        
        
        target=test_y_result
        prediction=test_predict_result
        print(target)
        print(prediction)
        error = []
        for i in range(min(len(target),len(prediction))):
            error.append(target[i] - prediction[i])
        squaredError = []
        absError = []
        for val in error:
            squaredError.append(val * val)#target-prediction之差平方 
            absError.append(abs(val))#误差绝对值
        return sqrt(sum(squaredError) / len(squaredError))
        #print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#均方根误差RMSE
        #plt.figure()
        #plt.plot(list(range(len(test_predict))), test_predict, color='b')
        #plt.plot(list(range(len(test_y))), test_y,  color='r')
        #plt.show()


prediction()

c:\users\lenovo\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


INFO:tensorflow:Restoring parameters from .\prediction.model-19
[-0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011  6.44204936 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011  6.44204936 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011
 -0.15523011 -0.15523011 -0.15523011 -0.15523011 -0.15523011

1.0057921754379058

In [ ]:
for 